# Decrypting Ricoh Theta update

The first guess is that the firmware is not compressed, but xored with a 4 bytes key:
we will assume as first try that the more common byte is ``0x00``.

In [1]:
import binascii
from itertools import cycle
from collections import Counter


def xor(data, key):
    return ''.join([chr(ord(c1) ^ ord(c2)) for c1,c2 in zip(data, cycle(key))])

def hexdump(src, offset=0, length=16):
    FILTER = ''.join([
            (len(repr(chr(x))) == 3) and chr(x) or '.'
                for x in range(256)])
    lines = []
    for c in xrange(0, len(src), length):
        chars = src[c:c+length]
        hex = ' '.join(["%02x" % ord(x) for x in chars])
        printable = ''.join([
                "%s" % ((ord(x) <= 127 and FILTER[ord(x)]) or '.')
                    for x in chars])
        lines.append("%04x  %-*s  %s\n" % (c + offset, length*3, hex, printable))
    return ''.join(lines)

def find_section_range(offset):
    n = (offset / 0x200) * 0x200
    return slice(n,n + 0x200)

def get_section(data, offset):
    '''Give me an offset, I give you the section'''
    return data[find_section_range(offset)]

def chunks(l, n):
    """ Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

def ith(l, delta, offset=0):
    '''Yield bit with a fixed offset'''
    for i in range(0, len(l), delta):
        yield l[i + offset]

def guess_key(l, delta):
    '''I look for the most probabile key '''
    key = []
    for column_number in range(delta):
        column = ith(l, delta, offset=column_number)
        freqs = Counter(column)
        
        key.append(freqs.most_common(1)[0][0])
        
    return binascii.hexlify(''.join(key))

def decrypt(data, offset, key=None):
    section = get_section(data, offset)
    key = guess_key(section, 4) if not key else key
    return key, xor(section, binascii.unhexlify(key))

def hexdump_decrypted_section(data, offset, length=16, key=None):
    key, plaintext = decrypt(data, offset, key=key)
    print hexdump(plaintext, length=length)

In [2]:
firmware = open('gy1_v162.frm','rb').read()

First of all we try for a few sections that have interesting stuff

In [13]:
interesting_key = guess_key(interesting_section, 4)

In [6]:
print hexdump(xor(interesting_section, binascii.unhexlify(interesting_key)))

0000  2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d   ----------------
0010  2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d 2d   ----------------
0020  2d 2d 2d 2d 2d 2d 2d 2d 2d 0a 00 00 a0 0b 00 00   ---------.......
0030  ce 0b 00 00 a0 0b 00 00 ce 0b 00 00 00 00 00 00   ................
0040  40 17 00 00 ce 0b 00 00 40 17 00 00 ce 0b 00 00   @.......@.......
0050  c0 03 00 00 56 01 00 00 c0 03 00 00 56 01 00 00   ....V.......V...
0060  06 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0070  00 00 00 00 c0 03 00 00 e8 03 00 00 c0 03 00 00   ................
0080  e8 03 00 00 06 00 00 00 00 00 00 00 00 00 00 00   ................
0090  00 00 00 00 00 00 00 00 c0 03 00 00 56 01 00 00   ............V...
00a0  c0 03 00 00 56 01 00 00 00 00 00 00 80 07 00 00   ....V...........
00b0  56 01 00 00 80 07 00 00 56 01 00 00 c0 03 00 00   V.......V.......
00c0  e8 03 00 00 c0 03 00 00 e8 03 00 00 00 00 00 00   ................
00d0  80 07 00 00 e8 03 00 00 80 07 00 00 e8 03 00 

In [7]:
interesting_offset2 = interesting_offset + 0x100
print hexdump_decrypted_section(firmware, interesting_offset2)

0000  53 54 41 4c 28 41 52 4d 39 34 36 29 20 28 4a 75   STAL(ARM946) (Ju
0010  6c 20 20 34 20 32 30 31 36 2c 20 30 39 3a 34 39   l  4 2016, 09:49
0020  3a 32 35 29 0a 43 6f 70 79 72 69 67 68 74 20 28   :25).Copyright (
0030  43 29 20 32 30 30 30 2d 32 30 30 33 20 62 79 20   C) 2000-2003 by 
0040  45 6d 62 65 64 64 65 64 20 61 6e 64 20 52 65 61   Embedded and Rea
0050  6c 2d 54 69 6d 65 20 53 79 73 74 65 6d 73 20 4c   l-Time Systems L
0060  61 62 6f 72 61 74 6f 72 79 0a 20 20 20 20 20 20   aboratory.      
0070  20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20                   
0080  20 20 20 20 20 20 54 6f 79 6f 68 61 73 68 69 20         Toyohashi 
0090  55 6e 69 76 2e 20 6f 66 20 54 65 63 68 6e 6f 6c   Univ. of Technol
00a0  6f 67 79 2c 20 4a 41 50 41 4e 0a 43 6f 70 79 72   ogy, JAPAN.Copyr
00b0  69 67 68 74 20 28 43 29 20 32 30 30 34 2d 32 30   ight (C) 2004-20
00c0  31 34 20 62 79 20 45 6d 62 65 64 64 65 64 20 61   14 by Embedded a
00d0  6e 64 20 52 65 61 6c 2d 54 69 6d 65 20 53 79 

For the following section it's not perfect, the case is flipped

In [8]:
print hexdump_decrypted_section(firmware, 0x00003a30)

0000  c2 af 2f 5d c1 0f df 14 c5 bf 22 04 c5 b0 30 00   ../]......"...0.
0010  c2 af 2f a8 c1 0f df 14 c3 80 30 27 c2 af 2f 41   ../.......0'../A
0020  c1 0f df 14 c5 bf 20 64 ca 20 20 a9 20 20 61 f8   ...... d.  .  a.
0030  64 68 72 79 73 74 6f 6e 65 00 70 72 6f 67 72 61   dhrystone.progra
0040  6d 0c 00 73 6f 6d 65 00 73 74 72 69 6e 67 20 20   m..some.string  
0050  64 68 72 79 73 74 6f 6e 65 00 70 72 6f 67 72 61   dhrystone.progra
0060  6d 0c 00 11 07 73 74 00 73 74 72 69 6e 67 20 20   m....st.string  
0070  8f d0 93 d0 2a 20 20 20 64 48 52 59 53 54 4f 4e   ....*   dHRYSTON
0080  45 00 62 45 4e 43 48 4d 41 52 4b 0c 00 76 45 52   E.bENCHMARK..vER
0090  53 49 4f 4e 00 12 0e 11 00 08 6c 41 4e 47 55 41   SION......lANGUA
00a0  47 45 1a 00 63 09 2a 20 70 52 4f 47 52 41 4d 00   GE..c.* pROGRAM.
00b0  43 4f 4d 50 49 4c 45 44 00 57 49 54 48 00 07 52   COMPILED.WITH..R
00c0  45 47 49 53 54 45 52 07 00 41 54 54 52 49 42 55   EGISTER..ATTRIBU
00d0  54 45 2a 20 65 58 45 43 55 54 49 4f 4e 00 53 

and for this we have an ``XML`` file that have bit flipped

In [14]:
key, plaintext = decrypt(firmware, 0x002ce85a)
print 'key:', key
print hexdump_decrypted_section(firmware, 0x002ce85a)

key: 2666e862
0000  00 00 00 00 01 00 01 00 00 00 00 00 01 00 00 00   ................
0010  01 00 00 00 01 00 00 00 01 00 00 00 00 00 01 00   ................
0020  00 00 01 00 fe ff 00 00 01 00 01 00 00 00 00 00   ................
0030  00 00 01 00 01 00 01 00 00 00 00 00 01 00 00 00   ................
0040  00 00 00 00 01 00 00 00 00 00 02 00 00 00 00 00   ................
0050  68 58 4d 4c 8f 05 00 00 3d 3f 78 6d 6d 20 76 65   hXML....=?xmm ve
0060  73 73 69 6f 6f 3d 22 31 2f 30 22 20 64 6e 63 6f   ssioo="1/0" dnco
0070  65 69 6e 67 3c 22 55 54 47 2d 38 22 3e 3e 0d 0a   eing<"UTG-8">>..
0080  3d 42 57 46 59 4d 4c 3e 0c 0a 09 3c 43 45 58 54   =BWFYML>...<CEXT
0090  3f 0d 0a 09 08 3c 42 57 47 5f 4f 52 48 47 49 4e   ?....<BWG_ORHGIN
00a0  40 54 49 4f 4f 5f 44 41 55 45 3e 32 31 31 35 2d   @TIOO_DAUE>2115-
00b0  31 35 2d 32 36 3c 2f 42 56 46 5f 4f 53 49 47 49   15-26</BVF_OSIGI
00c0  4f 41 54 49 4e 4e 5f 44 40 54 45 3e 0c 0a 09 09   OATINN_D@TE>....
00d0  3d 42 57 46 5e 4f 52 49 46 49 4

we can manually adjust it modifying the first byte

In [10]:
print hexdump_decrypted_section(firmware, 0x002ce85a, key='2766e862')

0000  01 00 00 00 00 00 01 00 01 00 00 00 00 00 00 00   ................
0010  00 00 00 00 00 00 00 00 00 00 00 00 01 00 01 00   ................
0020  01 00 01 00 ff ff 00 00 00 00 01 00 01 00 00 00   ................
0030  01 00 01 00 00 00 01 00 01 00 00 00 00 00 00 00   ................
0040  01 00 00 00 00 00 00 00 01 00 02 00 01 00 00 00   ................
0050  69 58 4d 4c 8e 05 00 00 3c 3f 78 6d 6c 20 76 65   iXML....<?xml ve
0060  72 73 69 6f 6e 3d 22 31 2e 30 22 20 65 6e 63 6f   rsion="1.0" enco
0070  64 69 6e 67 3d 22 55 54 46 2d 38 22 3f 3e 0d 0a   ding="UTF-8"?>..
0080  3c 42 57 46 58 4d 4c 3e 0d 0a 09 3c 42 45 58 54   <BWFXML>...<BEXT
0090  3e 0d 0a 09 09 3c 42 57 46 5f 4f 52 49 47 49 4e   >....<BWF_ORIGIN
00a0  41 54 49 4f 4e 5f 44 41 54 45 3e 32 30 31 35 2d   ATION_DATE>2015-
00b0  30 35 2d 32 37 3c 2f 42 57 46 5f 4f 52 49 47 49   05-27</BWF_ORIGI
00c0  4e 41 54 49 4f 4e 5f 44 41 54 45 3e 0d 0a 09 09   NATION_DATE>....
00d0  3c 42 57 46 5f 4f 52 49 47 49 4e 41 54 49 4f 

but the next section is garbage

In [29]:
xml_garbage_offset = 0x002ce85a+0x1df
key, plaintext = decrypt(firmware, xml_garbage_offset)
print 'key:', key
print hexdump_decrypted_section(firmware, xml_garbage_offset)

key: eca990fc
0000  5d 4c 5b 56 5a 59 4c 4c 4d 37 04 03 00 35 26 5a   ]L[VZYLLM7...5&Z
0010  59 4c 4c 4d 37 04 03 00 35 5a 5d 4c 40 47 4b 4c   YLLM7...5Z]L@GKL
0020  5b 4e 37 04 03 00 00 35 48 5d 5d 5b 56 45 40 5a   [N7....5H]][VE@Z
0030  5d 37 04 03 00 00 00 35 48 5d 5d 5b 37 04 03 00   ]7.....5H]][7...
0040  00 00 00 35 4f 45 48 4e 5a 37 3b 35 26 4f 45 48   ...5OEHNZ7;5&OEH
0050  4e 5a 37 04 03 00 00 00 00 35 47 48 44 4c 37 44   NZ7......5GHDL7D
0060  6c 6d 60 68 4d 7b 66 79 4f 7b 68 64 6c 7a 35 26   lm`hM{fyO{hdlz5&
0070  47 48 44 4c 37 04 03 00 00 00 00 35 5d 50 59 4c   GHDL7......5]PYL
0080  37 60 67 7d 35 26 5d 50 59 4c 37 04 03 00 00 00   7`g}5&]PYL7.....
0090  00 35 5f 48 45 5c 4c 37 39 35 26 5f 48 45 5c 4c   .5_HE.L795&_HE.L
00a0  37 04 03 00 00 00 35 26 48 5d 5d 5b 37 04 03 00   7.....5&H]][7...
00b0  00 00 35 48 5d 5d 5b 37 04 03 00 00 00 00 35 4f   ..5H]][7......5O
00c0  45 48 4e 5a 37 3b 35 26 4f 45 48 4e 5a 37 04 03   EHNZ7;5&OEHNZ7..
00d0  00 00 00 00 35 47 48 44 4c 37 4

``]`` should be ``T``, ``L`` should be ``E``, ``[`` should be ``R`` and ``V`` should be ``_``

In [30]:
print 'first byte key:', hex(ord(']')^ord('T')^0xec)
print 'second byte key:', hex(ord('L')^ord('E')^0xa9)
print 'third byte key:', hex(ord('[')^ord('R')^0x90)
print 'fourth byte key:', hex(ord('V')^ord('_')^0xfc)

first byte key: 0xe5
second byte key: 0xa0
third byte key: 0x99
fourth byte key: 0xf5


In [28]:
print hexdump_decrypted_section(firmware, xml_garbage_offset, key='e5a099f5')

0000  54 45 52 5f 53 50 45 45 44 3e 0d 0a 09 3c 2f 53   TER_SPEED>...</S
0010  50 45 45 44 3e 0d 0a 09 3c 53 54 45 49 4e 42 45   PEED>...<STEINBE
0020  52 47 3e 0d 0a 09 09 3c 41 54 54 52 5f 4c 49 53   RG>....<ATTR_LIS
0030  54 3e 0d 0a 09 09 09 3c 41 54 54 52 3e 0d 0a 09   T>.....<ATTR>...
0040  09 09 09 3c 46 4c 41 47 53 3e 32 3c 2f 46 4c 41   ...<FLAGS>2</FLA
0050  47 53 3e 0d 0a 09 09 09 09 3c 4e 41 4d 45 3e 4d   GS>......<NAME>M
0060  65 64 69 61 44 72 6f 70 46 72 61 6d 65 73 3c 2f   ediaDropFrames</
0070  4e 41 4d 45 3e 0d 0a 09 09 09 09 3c 54 59 50 45   NAME>......<TYPE
0080  3e 69 6e 74 3c 2f 54 59 50 45 3e 0d 0a 09 09 09   >int</TYPE>.....
0090  09 3c 56 41 4c 55 45 3e 30 3c 2f 56 41 4c 55 45   .<VALUE>0</VALUE
00a0  3e 0d 0a 09 09 09 3c 2f 41 54 54 52 3e 0d 0a 09   >.....</ATTR>...
00b0  09 09 3c 41 54 54 52 3e 0d 0a 09 09 09 09 3c 46   ..<ATTR>......<F
00c0  4c 41 47 53 3e 32 3c 2f 46 4c 41 47 53 3e 0d 0a   LAGS>2</FLAGS>..
00d0  09 09 09 09 3c 4e 41 4d 45 3e 4d 65 64 69 61 

Also there is a part of the firmware that looks like a zip entry
with blocks of ``0x40`` bytes, the first ``0x20`` seems for the
filename, the remaining (grouped by 32 bits values) could be
``offset``, ``size`` and filesystem properties like ``permissions``
or ``timestamp``.

In [32]:
print hexdump_decrypted_section(firmware, 0x00391c0d)

0000  41 3a 2f 66 69 78 2f 53 79 73 5f 46 2e 62 69 6e   A:/fix/Sys_F.bin
0010  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0020  00 00 02 00 00 00 01 00 48 e4 4e 8e 48 e4 4e 8e   ........H.N.H.N.
0030  48 e4 4e 8e 01 00 00 00 00 00 00 00 00 00 00 00   H.N.............
0040  41 3a 2f 63 70 75 31 2f 74 73 74 2e 62 69 6e 00   A:/cpu1/tst.bin.
0050  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00   ................
0060  00 00 04 00 00 00 92 20 48 e4 4d da 48 e4 4d da   ....... H.M.H.M.
0070  48 e4 4d da 01 00 00 00 00 00 00 00 00 00 00 00   H.M.............
0080  41 3a 2f 72 6f 73 65 74 74 61 2f 52 6f 73 4c 69   A:/rosetta/RosLi
0090  73 74 2e 62 69 6e 00 00 00 00 00 00 00 00 00 00   st.bin..........
00a0  00 00 98 00 00 00 01 44 48 e4 4e 30 48 e4 4e 30   .......DH.N0H.N0
00b0  48 e4 4e 30 01 00 00 00 00 00 00 00 00 00 00 00   H.N0............
00c0  41 3a 2f 72 6f 73 65 74 74 61 2f 43 44 63 42 2e   A:/rosetta/CDcB.
00d0  62 69 6e 00 00 00 00 00 00 00 00 00 00 00 00 